# Project implementation and note

## Code Block Explanations

**Imports and Constants:** This section imports the necessary modules for the expense tracker application. The `os` module provides operating system interface functions, particularly used here to check file existence with `os.path.exists()`. The `datetime` module is imported to handle date operations, specifically using `datetime.now().strftime('%Y-%m-%d')` to generate current date strings in YYYY-MM-DD format. The `FILE_NAME` constant is set to "expenses.txt", which serves as the persistent storage file for all expense records. This file-based approach allows data to persist between program runs, making it a simple database alternative for this application.

In [ ]:
import os
from datetime import datetime

FILE_NAME = "expenses.txt"

**add_expense Function:** This function handles adding new expense entries to the tracking system. It accepts a category and amount as required parameters, plus optional keyword arguments through `**kwargs`. The date parameter uses a default value mechanism - if no date is provided, it automatically uses the current date formatted as YYYY-MM-DD. The function constructs a CSV-formatted entry string with date, category, and amount separated by commas. It opens the expenses file in append mode ('a'), ensuring new entries are added without overwriting existing data. After writing the entry, it provides user feedback by printing the added expense details. This function forms the core data input mechanism for the expense tracking system.

In [ ]:
def add_expense(category, amount, **kwargs):
    date = kwargs.get('date',datetime.now().strftime('%Y-%m-%d'))
    entry = f"{date},{category},{amount}\n"
    with open(FILE_NAME, 'a' ) as file:
        file.write(entry)
    print(f"Added:{category} - ${amount} on {date}")

**view_expenses Function:** This function displays all recorded expenses to the user. It first performs a safety check using `os.path.exists()` to verify the expenses file exists before attempting to read it. If the file doesn't exist, it informs the user that no expenses have been recorded yet and returns early to prevent errors. When the file exists, it opens it in read mode and uses `readlines()` to get all lines as a list. The function iterates through each line, using `strip()` to remove trailing newlines and whitespace, then prints each expense entry. This provides users with a complete view of their expense history in chronological order as stored in the file.

In [ ]:
def view_expenses():
    if not os.path.exists(FILE_NAME):
        print("No expenses recorded yet.")
        return
    with open(FILE_NAME, 'r') as file:
        lines = file.readlines()
        for _ in lines:
            print(_.strip())

**filter_expenses Function:** This function enables users to view expenses filtered by a specific category. It opens the expenses file and reads all lines, then uses a list comprehension to filter entries. The filtering logic splits each line by commas, accesses the category field (index 1), strips whitespace, and performs a case-insensitive comparison with the target category using `.lower()`. This ensures "Food" and "food" are treated as the same category. The filtered results are stored in a list and then displayed to the user with a descriptive header. This function is particularly useful for users who want to analyze spending patterns within specific categories like groceries, entertainment, or transportation.

In [ ]:
def filter_expenses(category):
    with open(FILE_NAME, 'r') as file:
        filtered = [line.strip() for line in file if line.split(',')[1].strip().lower() == category.lower()]
    print(f"\nFiltered expenses for category '{category}':")
    for line in filtered:
        print(line)

**expense_stats Function:** This function provides statistical analysis of recorded expenses. It begins with a file existence check similar to other functions. The function contains two nested helper functions: `parse()` extracts amounts from all expense entries by splitting each line and converting the third field (amount) to a float, and `stats()` calculates total and average from the amounts list. The main function calls these helpers in sequence, first parsing all amounts, then calculating statistics. It handles the edge case of empty data by setting average to 0 when no amounts exist. Finally, it displays the total and average expenses formatted to two decimal places, providing users with valuable insights into their spending patterns and financial overview.

In [ ]:
def expense_stats():
    if not os.path.exists(FILE_NAME):
        print("No data available")
        return
    
    def parse():
        with open(FILE_NAME, 'r') as file:
            return [float(line.split(',')[2]) for line in file]
        
    def stats(amounts):
        total = sum(amounts)
        if amounts:
            average = total / len(amounts)
        else:
            average = 0
        return total, average
    
    amounts = parse()
    total, avg = stats(amounts)
    print(f"\nTotal Expenses: ${total:.2f} | Average: ${avg:.2f}")

**remove_expense Function:** This function allows users to delete specific expense entries from their records. It accepts date and category parameters to identify which entries to remove. After checking file existence, it reads all existing lines into memory. The core logic uses a list comprehension with conditional filtering - it keeps lines that don't match both the specified date (using `startswith()`) and category (using `in` operator for substring matching). The filtered list excludes matching entries, effectively removing them. The function then writes the updated list back to the file in write mode ('w'), which overwrites the original content. This approach ensures data integrity while allowing selective deletion of expense records.

In [ ]:
def remove_expense(date, category):
    if not os.path.exists(FILE_NAME):
        print("File not found")
        return
    with open(FILE_NAME, 'r') as file:
        lines = file.readlines()

    updated = [line for line in lines if not (line.startswith(date) and category in line)]
    with open (FILE_NAME, 'w') as file:
        file.writelines(updated)

    print(f"Removed entries with date: {date} and category: {category}")

**list_categories Function:** This utility function helps users see what expense categories they've used previously. It reads the expenses file and extracts the category field from each line by splitting on commas and accessing index 1. The function uses a set comprehension to collect unique categories, automatically eliminating duplicates while converting all categories to lowercase for consistency. The `set` data structure ensures each category appears only once, regardless of how many times it was used. The function then displays these unique categories to the user, helping them maintain consistency when adding new expenses and providing insight into their spending categorization patterns.

In [ ]:
def list_categories():
    with open(FILE_NAME, 'r') as file:
        categories = set(line.split(',')[1].strip().lower() for line in file)
    print("\nCategories Used:", categories)

**main Function:** This function implements the interactive command-line interface for the expense tracker. It runs an infinite loop displaying a numbered menu of available operations. The menu includes options for adding expenses, viewing all records, filtering by category, showing statistics, removing entries, listing categories, and exiting. The function uses `input()` to capture user choices and employs a series of if-elif statements to execute corresponding functionality. For each option, it prompts for necessary input parameters and calls the appropriate function. The loop continues until the user selects option 7 (Exit), at which point it breaks the loop and terminates the program gracefully. 

In [ ]:
def main():
    while True:
        print("""
-------- Expense Tracker Menu --------
1. Add Expense
2. View All
3. Filter by Category
4. Show Stats
5. Remove Entry
6. Show Categories
7. Exit
        """)
        choice = input("Choose option: ")

        if choice == '1':
            cat = input("Enter category: ")
            amt = float(input("Enter amount ($): "))
            add_expense(cat, amt)
        elif choice == '2':
            view_expenses()
        elif choice == '3':
            cat = input("Enter category to filter: ")
            filter_expenses(cat)
        elif choice == '4':
            expense_stats()
        elif choice == '5':
            dt = input("Enter date (YYYY-MM-DD): ")
            cat = input("Enter category: ")
            remove_expense(dt, cat)
        elif choice == '6':
            list_categories()
        elif choice == '7':
            print("Exiting Expense Logger.")
            break
        else:
            print("Invalid option. Try again.")

**Entry Point Check:** This standard Python idiom ensures the main function only executes when the script is run directly, not when imported as a module. The `if __name__ == '__main__':` condition checks if the script is being executed as the main program. When true, it calls the `main()` function to start the interactive expense tracker interface. This pattern is a Python best practice that makes the code more modular and reusable, allowing the functions to be imported into other scripts without automatically starting the interactive interface.

In [ ]:
if __name__ == '__main__':
    main()